In [1]:
import requests
import csv
import os
import boto3
from datetime import datetime, timedelta

In [18]:
start_date = datetime.strptime("01012021", "%d%m%Y")
end_date = datetime.strptime("31122021", "%d%m%Y")
data_list = []

while start_date <= end_date:
    date_str = start_date.strftime("%d%m%Y")
    url = f"https://bank.gov.ua/NBU_Exchange/exchange?date={date_str}&json"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        data_list.extend(data)
    else:
        print(f"Error on {date_str}: {response.status_code}")

    # Збільшити дату на один день
    start_date += timedelta(days=1)

KeyboardInterrupt: 

In [ ]:
with open("exchange_rates_2021.csv", "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['StartDate', 'TimeSign', 'CurrencyCode', 'CurrencyCodeL', 'Units', 'Amount']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Записати заголовки CSV-файлу
        writer.writeheader()

        # Записати дані в CSV-файл
        for row in data_list:
            writer.writerow(row)

In [ ]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
csv_file_path = "exchange_rates_2021.csv" 
s3_bucket_name = "bucket-kostia"

upload_file(csv_file_path, s3_bucket_name)